<a href="https://colab.research.google.com/github/muxueya/HateDetectorHatescanAPI/blob/main/HateDetectorHatescanAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Step 1: Install Required Libraries
!pip install krippendorff scikit-learn pandas openpyxl



In [12]:
# Step 2: Import Libraries
import requests
import json
import csv
import pandas as pd
import re
import krippendorff
from sklearn.metrics import accuracy_score
import numpy as np



In [13]:
# Step 3: Define Function to Get Toxic Probability from Hatescan API
def get_toxic_probability(text, language='en', flag_detect_lang='False'):
    # Prepare the payload
    text = text.replace('"', '\\"')
    payload = '{"text": "' + text + '", "language": "' + language + '" , "flag_detect_lang": "' + flag_detect_lang + '"}'
    json_payload = json.loads(payload, strict=False)

    # Headers for the API request
    headers = {"Content-Type": "application/json; charset=utf-8"}

    # Hatescan API URL (updated URL)
    api_hatescan_url = 'https://api.hatescan.com/predict/toxic'

    # Send a POST request to the Hatescan API
    hatescan_response = requests.post(api_hatescan_url, headers=headers, json=json_payload)

    # Return the API response as JSON
    return hatescan_response.json()

# Step 4: Define Function to Read Comments from Excel File
def read_comments_from_excel(filename):
    # Load the comments from an Excel file (assuming the column name is 'comment')
    df = pd.read_excel(filename)
    comments = df['comment'].astype(str).tolist()  # Convert comments to a list of strings
    return comments



In [ ]:
# # Step 5: Process Comments for Toxicity Detection
# def process_comments(input_file, output_file, language='en', flag_detect_lang='False'):
#     # Read comments from the Excel file
#     comments = read_comments_from_excel(input_file)
#     num_comments = len(comments)

#     # Prepare the result list
#     results = []

#     # Loop through each comment and get toxicity scores
#     for idx, comment in enumerate(comments, 1):
#         print(f"Processing comment {idx}/{num_comments}: {comment[:100]}")
#         try:
#             toxic_score = get_toxic_probability(comment, language, flag_detect_lang)

#             # Correct key to access predictions
#             toxic_predictions = toxic_score.get('predictions', 'N/A')

#             # Output the results to the terminal
#             print(f"Toxic probability score: {toxic_predictions}%\n")

#             # Append the comment and toxic probabilities to the result list
#             results.append({
#                 'comment': comment,
#                 'toxic_predictions': toxic_predictions
#             })
#         except Exception as e:
#             print(f"Error processing comment: {comment}, Error: {e}")

#     # Save the results to a CSV file
#     with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
#         fieldnames = ['comment', 'toxic_predictions']
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#         writer.writeheader()
#         writer.writerows(results)

#     print(f"Results saved to {output_file}")

# # Usage Example
# input_file = 'incels-5000.xlsx'  # Replace with your actual file path
# output_file = 'hatescan_results.csv'
# process_comments(input_file, output_file)



In [ ]:
# # Step 6: Sample Comments with High and Low Toxicity
# def select_comments(input_file, high_output_file, low_output_file, X, Y):
#     # Load the toxicity results
#     df = pd.read_csv(input_file)

#     # Convert 'toxic_predictions' column to numeric
#     df['toxic_predictions'] = pd.to_numeric(df['toxic_predictions'], errors='coerce')

#     # Filter for high and low toxicity comments
#     high_toxic = df[df['toxic_predictions'] >= 50]
#     low_toxic = df[df['toxic_predictions'] < 50]

#     # Randomly select X and Y comments
#     selected_high_toxic = high_toxic.sample(n=X, random_state=42)
#     selected_low_toxic = low_toxic.sample(n=Y, random_state=42)

#     # Save the high-toxicity comments to a CSV file
#     selected_high_toxic.to_csv(high_output_file, index=False, encoding='utf-8')

#     # Save the low-toxicity comments to a separate CSV file
#     selected_low_toxic.to_csv(low_output_file, index=False, encoding='utf-8')

#     print(f"Randomly selected {X} high-toxicity comments saved to {high_output_file}")
#     print(f"Randomly selected {Y} low-toxicity comments saved to {low_output_file}")

# # Usage Example
# select_comments('hatescan_results.csv', 'high_toxic_comments.csv', 'low_toxic_comments.csv', X=10, Y=10)



In [14]:
# Step 7: Calculate Krippendorff's Alpha for Annotators
def calculate_krippendorff_alpha(file):
    df = pd.read_excel(file)
    annotations = df[['a', 'b', 'c', 'd', 'e']].values.T  # Transpose the data for Krippendorff's alpha
    alpha = krippendorff.alpha(reliability_data=annotations, level_of_measurement='nominal')
    print(f" {file} Krippendorff's Alpha: {alpha:.3f}")

# Example Usage
calculate_krippendorff_alpha('Sample_high_toxic_comments.xlsx')
calculate_krippendorff_alpha('Sample_low_toxic_comments.xlsx')
calculate_krippendorff_alpha('Sample_toxic_comments.xlsx')


 Sample_high_toxic_comments.xlsx Krippendorff's Alpha: 0.365
 Sample_low_toxic_comments.xlsx Krippendorff's Alpha: 0.511
 Sample_toxic_comments.xlsx Krippendorff's Alpha: 0.615


In [17]:
# Step 8: Calculate Accuracy for Dictionary and ML Methods
def calculate_accuracy(file):
    df = pd.read_excel(file)  # Load annotated samples

    # Calculate ML Method Accuracy
    ml_accuracy = accuracy_score(df['label2'], df['label1'])
    print(f"ML Method Accuracy: {ml_accuracy:.3f}")

# Example Usage
calculate_accuracy('Sample_toxic_comments - Copy.xlsx')


ML Method Accuracy: 0.690
